In [1]:
import pandas as pd

In [2]:
df_22 = pd.read_csv('2022rookiestats_TE.csv')
df_23 = pd.read_csv('2023rookiestats_TE.csv')
df_24 = pd.read_csv('2024rookiestats_TE.csv')
df_22_season = pd.read_csv('2022playerstats_TE.csv')
df_23_season = pd.read_csv('2023playerstats_TE.csv')
df_24_season = pd.read_csv('2024playerstats_TE.csv')

# Rookie TE Model:
- games played
- recs per game
- rec yards per game
- rec tds per game


In [3]:
def compute_features(df):
    df['GamesPlayed'] = df['Coll_games'].fillna(0)

    df['RecsPerGame'] = df['Coll_rec'] / df['Coll_games'].replace(0, pd.NA)
    df['RecsPerGame'] = df['RecsPerGame'].fillna(0)

    df['RecYdsPerGame'] = df['Coll_rec_yds_per_g'].fillna(0)

    df['RecTDsPerGame'] = df['Coll_rec_td'] / df['Coll_games'].replace(0, pd.NA)
    df['RecTDsPerGame'] = df['RecTDsPerGame'].fillna(0)

In [4]:
compute_features(df_22)
compute_features(df_23)
compute_features(df_24)

In [5]:
from helpers import append_total_fantasy_points

df_22_merged = append_total_fantasy_points(df_22, df_22_season)
df_23_merged = append_total_fantasy_points(df_23, df_23_season)
df_24_merged = append_total_fantasy_points(df_24, df_24_season)

In [6]:
features = [
    'GamesPlayed',
    'RecsPerGame',
    'RecYdsPerGame',
    'RecTDsPerGame'
]

df_22_23_merged = pd.concat([df_22_merged, df_23_merged], ignore_index=True)

In [7]:
y_test = df_24_merged['FantasyPtsPPR']
y_train = df_22_23_merged['FantasyPtsPPR']
X_test = df_24_merged[features]
X_train = df_22_23_merged[features]

In [8]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler().fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [9]:
# Diagnostics
print(f"Training samples: {len(X_train)}")
print(f"Testing samples: {len(X_test)}")
print(f"Features: {len(features)}")
print(f"Samples per feature: {len(X_train) / len(features):.1f}")
print(f"Training target stats: mean={y_train.mean():.1f}, std={y_train.std():.1f}")
print(f"Testing target stats: mean={y_test.mean():.1f}, std={y_test.std():.1f}")

Training samples: 32
Testing samples: 12
Features: 4
Samples per feature: 8.0
Training target stats: mean=45.6, std=51.7
Testing target stats: mean=49.6, std=73.0


In [10]:
from sklearn.linear_model import LinearRegression
model = LinearRegression().fit(X_train, y_train)

train_preds = model.predict(X_train)
preds = model.predict(X_test)

from helpers import evaluate_model
evaluate_model(y_train, train_preds, y_test, preds)

Training RMSE: 42.490583211710224
Training R^2: 0.302014915794133
Training MAE: 31.253446088486236
Testing RMSE: 62.10295463818111
Testing R^2: 0.20967370035905586
Testing MAE: 43.08028447685408


In [11]:
from helpers import print_model_coefficients
print_model_coefficients(model, features)

Intercept: 45.625
Feature Coefficients:
GamesPlayed: -1.2248
RecsPerGame: 5.7826
RecYdsPerGame: 31.9093
RecTDsPerGame: -20.7094


In [12]:
from helpers import compute_rank_squared_error, build_results_df
results = build_results_df(y_test, preds, df_24_merged['Player'])
rank_squared_error = compute_rank_squared_error(results)

print(results.to_string())
print("2024 Rank Squared Error:", rank_squared_error)


               Player  Actual  Predicted  ActualRank  PredictedRank  RankError
0        Brock Bowers   262.7  94.323243           1              1          0
1         Ben Sinnott    13.8  55.506248           7              5         -2
2          Tip Reiman     9.7  18.972874           9             11          2
3   Ja'Tavion Sanders    73.2  77.828196           3              3          0
4        Theo Johnson    68.1  29.562707           4             10          6
5           Erick All    35.8  53.203297           5              6          1
6           AJ Barner    78.5  32.071854           2              9          7
7         Cade Stover    34.3  40.724258           6              7          1
8         Jared Wiley     1.7  18.338952          11             12          1
9    Tanner McLachlan     0.0  90.195412          12              2        -10
10        Jaheim Bell     4.0  61.320035          10              4         -6
11         Devin Culp    13.8  33.831944           7